<a href="https://colab.research.google.com/github/Nicordaro/Project_MLDL/blob/master/Resnet_straana_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn


import torchvision
from torchvision import transforms
from torchvision.models import resnet18
from torchvision.models import resnet34
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
#from google.colab import drive

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

# Init at 10 because first train is on 10 classes
NUM_CLASSES = 10

# Used for the pseudorandom shuffle of the split
SEED = 42

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 0.045           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 1e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 70     # Total number of training epochs (iterations over dataset)
MILESTONES = [48, 62]  # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.2          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 50

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([  # Crops a central square patch of the image
                                     # 224 because torchvision's AlexNet needs a 224x224 inpu                           # Remember this when applying different transformations, otherwise you get an error
                                      transforms.Pad(4),
                                      transforms.RandomCrop(32),
                                      transforms.RandomHorizontalflip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  # https://gist.github.com/weiaicunzai/e623931921efefd4c331622c344d8151
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))                                 
])

AttributeError: module 'torchvision.transforms' has no attribute 'RandomHorizontalflip'

In [0]:
# Clone github repository with data
# if os.path.isdir('./Project_MLDL'):
!rm -rf Project_MLDL
if not os.path.isdir('./CIFAR100_tError'):
  !git clone https://github.com/Nicordaro/Project_MLDL


In [0]:
from Project_MLDL.CIFAR100_tError import CIFAR100_tError
import numpy as np
import random

DATA_DIR = './CIFAR100'

lbls = [i for i in range(0,100)]  #Array of classes integer-encoded (?)
random.seed(SEED)
random.shuffle(lbls)

def make_data_labels(lbls):       #After shuffle, take first 10 classes, and remove the first 10 from the list passed as argument
    new_labels=[]
    for el in lbls[:10]:
        new_labels.append(el)
    lbls = lbls[10:]

    return lbls, new_labels

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [0]:
net = ResNet(BasicBlock, [2, 2, 2, 2],10)

In [0]:
import numpy as np
BATCH_TO_TEST = 10
accuracies = []

# Define test dataset outside in order to increment it, instead of initializing it every cycle iteration
test_dataset = CIFAR100_tError(DATA_DIR, train=False, transform=eval_transform, download=True)
for i in range(0,BATCH_TO_TEST): #one iteration for 10 classes
    print(f'Starting training with batch {i+1}')
    lbls, new_labels = make_data_labels(lbls)
  
  # Define Train dataset
    train_dataset = CIFAR100_tError(DATA_DIR, train=True, transform=train_transform, download=True)       
  
  # Increment dataset with new labels mapped with list comprehension
    train_dataset.increment(new_labels,[j for j in range(0+i*10,10+i*10)])
    test_dataset.increment(new_labels,[j for j in range(0+i*10,10+i*10)])

  # Change number of neurons in last fully connected layer
    net.linear = nn.Linear(512, NUM_CLASSES*(i+1))
#     if i>1:
#         new_block = nn.Linear(512, 10).to(DEVICE)
#         new_weights = torch.cat((net.fc.weight.data, new_block.weight.data), dim = 0)
#         new_biases = torch.cat((net.fc.bias.data, new_block.bias.data), dim = 0)

#         new_layer = nn.Linear(512 , new_weights.size(0))
#         net.fc = new_layer
#         net.fc.weight.data = new_weights
#         net.fc.bias.data = new_biases

  # Define dataloader
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

  #prepare training
  # Loss function
    criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

  # Parameters to optimize:
    parameters_to_optimize = net.parameters()

  # Optimizer
    optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  # Scheduler
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=GAMMA, last_epoch=-1)

  # training 
  # By default, everything is loaded to cpu
    net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

    cudnn.benchmark # Calling this optimizes runtime

    current_step = 0

  # Start iterating over the epochs
    for epoch in range(NUM_EPOCHS):
    # if epoch==5:
    #   for g in optimizer.param_groups:
    #     g['lr'] = g['lr']/5
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_last_lr()))
    
    # Iterate over the dataset
        for images, labels in train_dataloader:
      # Bring data over the device of choice
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            net.train() # Sets module in training mode

      # PyTorch, by default, accumulates gradients after each backward pass
      # We need to manually set the gradients to zero before starting a new iteration
            optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
            outputs = net(images)

      # Compute loss based on output and ground truth
            loss = criterion(outputs, labels)
      
            if current_step % LOG_FREQUENCY == 0:
                print('Step {}, Loss {}'.format(current_step, loss.item()))

      # Compute gradients for each layer and update weights
            loss.backward()  # backward pass: computes gradients
            optimizer.step() # update weights based on accumulated gradients
            current_step += 1

    # Step the scheduler
        scheduler.step()

  #test phase
    net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
    net.train(False) # Set Network to evaluation mode

    running_corrects = 0

    for images, labels in tqdm(test_dataloader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

    # Forward Pass
        outputs = net(images)

    # Get predictions
        _, preds = torch.max(outputs.data, 1)

    #Debugging purpose, print labels of predictions
    ##print(preds)

    # Update Corrects
        running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
    accuracy = running_corrects / float(len(test_dataset))

  #Store accuracies for plotting purposes
    accuracies.append(accuracy)
    print('Test Accuracy: {}'.format(accuracy))

Files already downloaded and verified
Starting training with batch 1
Files already downloaded and verified
Starting epoch 1/70, LR = [0.05]
Step 0, Loss 2.336782932281494
Starting epoch 2/70, LR = [0.05]
Step 50, Loss 1.8893650770187378
Starting epoch 3/70, LR = [0.05]
Step 100, Loss 1.8162317276000977
Starting epoch 4/70, LR = [0.05]
Step 150, Loss 1.547146201133728
Starting epoch 5/70, LR = [0.05]
Starting epoch 6/70, LR = [0.05]
Step 200, Loss 1.2183527946472168
Starting epoch 7/70, LR = [0.05]
Step 250, Loss 1.30564284324646
Starting epoch 8/70, LR = [0.05]
Step 300, Loss 1.119005799293518
Starting epoch 9/70, LR = [0.05]
Step 350, Loss 1.1366451978683472
Starting epoch 10/70, LR = [0.05]
Starting epoch 11/70, LR = [0.05]
Step 400, Loss 0.8038743734359741
Starting epoch 12/70, LR = [0.05]
Step 450, Loss 1.189079999923706
Starting epoch 13/70, LR = [0.05]
Step 500, Loss 1.1796729564666748
Starting epoch 14/70, LR = [0.05]
Starting epoch 15/70, LR = [0.05]
Step 550, Loss 0.9464735984

100%|██████████| 8/8 [00:00<00:00, 14.47it/s]


Test Accuracy: 0.785
Starting training with batch 2
Files already downloaded and verified
Starting epoch 1/70, LR = [0.05]
Step 0, Loss 3.131760835647583
Starting epoch 2/70, LR = [0.05]
Step 50, Loss 0.8882323503494263
Starting epoch 3/70, LR = [0.05]
Step 100, Loss 0.7089924216270447
Starting epoch 4/70, LR = [0.05]
Step 150, Loss 0.6119722127914429
Starting epoch 5/70, LR = [0.05]
Starting epoch 6/70, LR = [0.05]


KeyboardInterrupt: 

In [0]:
print(accuracies)